In [ ]:
!pip install tpot


     |████████████████████████████████| 92kB 4.9MB/s 
     |████████████████████████████████| 163kB 11.0MB/s 
     |████████████████████████████████| 157.5MB 93kB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11957 sha256=f99eed98190a3c0a1dfca078b01167cb93bcc6b8e771769b0984c84f69239dbf
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit
  Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
ls

ratings_new.csv  sample_data/


In [ ]:
import pandas
import numpy

# example of auto-sklearn for the ml-100k regression dataset
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import RepeatedKFold
from tpot import TPOTRegressor

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# load dataset
url = 'ratings_new.csv'
dataframe = read_csv(url)

In [ ]:
dataframe

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [ ]:
# load dataset

# split into input and output elements
data = dataframe.values
data = data.astype('float32')
X, y = data[:, :-1], data[:, -1]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
X_test.shape

(33276, 2)

In [ ]:
X_test

array([[2.25000e+02, 3.80000e+02],
       [5.33000e+02, 1.03688e+05],
       [1.25000e+02, 1.49902e+05],
       ...,
       [4.14000e+02, 5.35190e+04],
       [1.56000e+02, 6.21800e+03],
       [3.51000e+02, 9.20480e+04]], dtype=float32)

In [ ]:
# tpot for ml-100k dataset

# define evaluation procedure
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=121)
# define search
model = TPOTRegressor(generations=5, population_size=50, scoring='neg_mean_absolute_error', cv=cv, verbosity=2, random_state=1, n_jobs=-2, max_time_mins=60)
# perform the search
model.fit(X_train, y_train)
# export the best model
model.export('tpot_ml_100k_best_model.py')



60.34 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: XGBRegressor(LinearSVR(input_matrix, C=5.0, dual=True, epsilon=0.1, loss=squared_epsilon_insensitive, tol=0.001), learning_rate=1.0, max_depth=4, min_child_weight=6, n_estimators=100, n_jobs=1, objective=reg:squarederror, subsample=0.9500000000000001, verbosity=0)


## Fitting the dataset with the exported pipeline

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import LinearSVR
from tpot.builtins import StackingEstimator
from xgboost import XGBRegressor
from tpot.export_utils import set_param_recursive



# Average CV score on the training set was: -0.7206681370735168
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=LinearSVR(C=5.0, dual=True, epsilon=0.1, loss="squared_epsilon_insensitive", tol=0.001)),
    XGBRegressor(learning_rate=1.0, max_depth=4, min_child_weight=6, n_estimators=100, n_jobs=1, objective="reg:squarederror", subsample=0.9500000000000001, verbosity=0)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 121)

# X_train, X_test, y_train, y_test

exported_pipeline.fit(X_train, y_train)
results = exported_pipeline.predict(X_test)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
from sklearn.metrics import mean_squared_error


rmse = mean_squared_error(y_test, results)

In [ ]:
rmse

0.849122

In [ ]:
from sklearn.metrics import mean_absolute_error


mae = mean_absolute_error(y_test, results)

In [ ]:
mae

0.7112007